In [1]:
import random
import re
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoConfig, AutoModelForSeq2SeqLM, MBartForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
import torch
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
import evaluate
import numpy as np
import os
from datasets import load_dataset, concatenate_datasets

In [ ]:
file_list = ["한영번역/전체 데이터/" + file for file in os.listdir('한영번역/전체 데이터/') if file.endswith('.json')]

In [ ]:
datasets = Dataset.from_json(file_list[0], field='data')
datasets = datasets.remove_columns([column_name for column_name in datasets.column_names if column_name not in ['en', 'ko']])

In [ ]:
for index in range(1, len(file_list)):
    dataset = Dataset.from_json(file_list[index], field='data')
    dataset = dataset.remove_columns([column_name for column_name in dataset.column_names if column_name not in ['en', 'ko']])
    datasets = concatenate_datasets([datasets, dataset])

In [ ]:
datasets = datasets.rename_columns({"ko": "SRC", "en": "TRG"})

In [ ]:
path = './한영번역/'
train_df = pd.read_csv(path+'/Translation_dataset_split.csv')

dataset = Dataset.from_pandas(train_df)


train_test_valid_dataset = dataset.train_test_split(test_size=3000, seed=42)
train_test_valid_dataset = DatasetDict({
    'train': train_test_valid_dataset['train'],
    'valid': train_test_valid_dataset['test']})

In [ ]:
train_test_valid_dataset['train'] = concatenate_datasets([train_test_valid_dataset['train'], datasets])

In [ ]:
train_test_valid_dataset.save_to_disk("AIHub_data_Total")

In [2]:
dataset = DatasetDict.load_from_disk('AIHub_data_Total')


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['SRC', 'TRG'],
        num_rows: 9690651
    })
    valid: Dataset({
        features: ['SRC', 'TRG'],
        num_rows: 3000
    })
})